# Introduction to Python Package SQL_runner

## Part 1: what is SQL_runner
SQL_runner is a Python library providing PostgreSQL adaptors and QA tools. It supports for parsing, splitting, formatting SQL scripts as well as builidng QA tools for either customized or recurring projects.

## Part 2: advantages of SQL_runner

- Multi-Data Environments: play around different data bases from multiple data environments on a single aggregated platform
- Convenience: everything on the same page. (SQL script, Python code, Excel, Manual Work, Comments, etc.)
- Markdown: easy to check what you are doing/ what you have done, easy for project transition 
- Efficiency: One click to run recurring projects, no manual work needed
- QA tools: easy to build up customized QA pipeline/tools for different projects.
- Powerfulness: build a bridge between Ptyhon and Greenplum/Redshift/SQL_server/Snowflake, which makes it possible to leverage the whole python world(time, calendar, pandas, numpy, matplotlib, etc.)

## Part 3: how to use SQL runner

### Step 0: Import SQL_runner package

In [2]:
import sql_runner as sr

### Step 1: Connect to multiple data bases in different data environments
#### 1.1 GreenPlum - PostgreSQL

In [5]:
db1 = sr.connect_db(daname = 'GP_db1', user = 'wenqi', password = 'xxx')

Greenplum Database is successfully connected


In [6]:
db2 = sr.connect_db(daname = 'GP_db2', uesr = 'wenqi', password = 'xxx')

Greenplum Database is successfully connected


In [7]:
db3 = sr.connect_db(daname = 'GP_db3', user = 'wenqi', password = 'xxx')

Greenplum Database is successfully connected


#### 1.2 SQL Server - MySQL

In [8]:
# Windows Authentication
db4 = ss.connect_sql_server(daname = 'SQL_Server_db1', authentication = 'windows')

SQL_Server Database is successfully connected


In [15]:
# SQL_Server Aunthentication
db5 = sr.connect_sql_server(dbname = 'SQL_Server_db2', authentication = 'SQL_Server', user = 'user1', password = 'xxx')

SQL_Server Database is successfully connected


#### 1.3 Redshift - Redshift SQL

In [16]:
db6 = sr.connect_redshift(user = 'wenqi', password = 'xxx', host = 'xxxxx.db.xxxx.xxx.comscore.com', port = '5439')

Redshift Database live is successfully connected


### Step 2: how to make replacement and run sql script

In [17]:
# Set up script path
folder = '\\\\csiadcai02\\Custom_Scripts\\wecheng_script_csiadcai02\\Demo\\'
demo_script_path = folder + 'Demo_sql.sql'
# Set up month_id
calendar_month = '201801'

In [22]:
sr.parse_replace_execute(sql = demo_script_path,
                         conn = db2,
                         replace_items={'@CALENDAR_MONTH': calendar_month,
                                        '@VERSION' : 'wenqi'},
                         print_script = False,
                         save_granted_public_table=False, 
                         save_granted_public_table_path='', 
                         output_last_query_string=False
                        )

1. The path of running script: \\csiadcai02\Custom_Scripts\wecheng_script_csiadcai02\Demo\Demo_sql.sql

needed_replace: @VERSION
replace_to: wenqi

needed_replace: @CALENDAR_MONTH
replace_to: 201801

2. Input file: \\csor2gpl04\incoming\wecheng\Demo\demo_input_201801.txt
3. Create public table: public.demo_country_table_201801_wenqi
4. Table comscore.country_lookup is being used
5. Grant table: public.demo_country_table_201801_wenqi
6. Output_file: \\csor2gpl04\incoming\wecheng\Demo\demo_out_201801.txt
7. Table public.demo_country_table_201801_wenqi is being used

YOU ARE GOOD TO MOVE ON!


### Step 3: how to run a single line of sql

In [48]:
sr.run_sql_query('''
select *
from public.demo_country_table_@CALENDAR_MONTH_@VERSION
limit 5
''',
                 conn = db2, 
                 replace_items={'@CALENDAR_MONTH': calendar_month,
                                      '@VERSION' : 'wenqi'})

,calendar_month,country_name,country_code
0,201801,Brazil,BR
1,201801,France,FR
2,201801,Italy,IT
3,201801,Spain,ES
4,201801,Canada,CA


### Step 4: how to loop through multiple months

In [23]:
# loop through 2 months(201801, 201802)
for month in range(201801,201803):
    sr.parse_replace_execute(sql = demo_script_path,
                            conn = db2,
                            replace_items={'@CALENDAR_MONTH': str(month),
                                          '@VERSION' : 'wenqi'})

1. The path of running script: \\csiadcai02\Custom_Scripts\wecheng_script_csiadcai02\Demo\Demo_sql.sql

needed_replace: @VERSION
replace_to: wenqi

needed_replace: @CALENDAR_MONTH
replace_to: 201801

2. Input file: \\csor2gpl04\incoming\wecheng\Demo\demo_input_201801.txt
3. Create public table: public.demo_country_table_201801_wenqi
4. Table comscore.country_lookup is being used
5. Grant table: public.demo_country_table_201801_wenqi
6. Output_file: \\csor2gpl04\incoming\wecheng\Demo\demo_out_201801.txt
7. Table public.demo_country_table_201801_wenqi is being used

YOU ARE GOOD TO MOVE ON!
1. The path of running script: \\csiadcai02\Custom_Scripts\wecheng_script_csiadcai02\Demo\Demo_sql.sql

needed_replace: @VERSION
replace_to: wenqi

needed_replace: @CALENDAR_MONTH
replace_to: 201802

2. Input file: \\csor2gpl04\incoming\wecheng\Demo\demo_input_201802.txt
3. Create public table: public.demo_country_table_201802_wenqi
4. Table comscore.country_lookup is being used
5. Grant table: public

### Step 5: QA tool 1 - compare with historical data
This function compares and find different values between 2 tables

In [69]:
sr.compare_gp_table(table1 = 'public.demo_country_table_201801_wenqi',
                   table2 = 'public.demo_country_table_201802_wenqi',
                   conn = db2,
                   columns = 'country_name, country_code',
                   table1_name = 'last_month',
                   table2_name = 'current_month').sort_values('country_name')

table1:last_month
table2:current_month

num_of_rows(last_month):9
num_of_rows(current_month):9
WARNING !!! CHANGES ARE AS BELOW !!!


,country_name,country_code,lable
0,Brazil,BR,last_month
10,China,CN,current_month
2,United Kingdom,GB,last_month
9,United Kingdom,UK,current_month


### Step 6: QA tool 2 - calculate deltas and alert big changes
This function calculate the changes for field total visitors between last month data and this month data. Then it list out the entities with big change( delta percentage > 20%)

In [36]:
ss.compare_delta(table = 'public.entities_thismonth_final',
                 base = 'public.entities_lastmonth_final',
                 com_col= 'web_name',
                 comp_col = 'total_uv',
                 conn = db2, 
                 level = 0.2,
                 level_col = ['total_uv'],
                 hide = False,
                 condition = 'where condition 1 and condition 2')

,web_name,total_uv,total_uv_base,delta_total_uv
12,HuffPost India,1776271.948502,1363249.451579,0.302969
14,HUFFINGTONPOST.DE,17028.957246,7328.424168,1.323686
42,HUFFINGTONPOST.COM.MX,4896.333771,9835.616896,-0.502183


## PART 4: Resources:  
- Python packaged used in the SQL_runner: sqlparse [click here](https://pypi.org/project/sqlparse/), psycopg2 [click here](http://initd.org/psycopg/), pandas, etc.
- sql scripts used in this demo [click here](https://github.com/wenqicheng/SQL_runner/blob/master/Demo_sql.sql)
- Wenqi's Linkedin [click here](https://www.linkedin.com/in/wenqicheng/)
- Wenqi's Github homepage [click here](https://github.com/wenqicheng)